PCA (Principal Component Analysis, Hauptkomponentenanalyse) ist ein Multivarianten Statistik Verfahren, welches umfangreiche Datensätze strukturiert, vereinfacht und veranschaulicht, indem eine Vielzahl statistischer Variablen durch eine geringere Zahl möglichst aussagekräftiger Linearkombinationen ("Hauptkomponenten") genähert wird. (Dimensionen Reduktion)

Wir bekommen "faces" Datensatz, welches fürs Training der Gesichts Erkennung abgebildet ist. Wir entscheiden uns für eine neue Anzahl an Dimensionenum, und reduzieren die Anzahl von Dimensionen.
(http://vis-www.cs.umass.edu/lfw/)

Zuerst packen wir alle Gesicht Images in einem Ordner ein, lese die nacheinander, gleichzeitig wandeln die (n * n) Array ins (1 * n) Array, damit jede Zeile zu einem Gesichtsbild in der Matrix entspricht.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

def get_file():
    path = 'Ressources/faces'
    files = os.listdir(path='Ressources/faces')
    s = []
    for file in files:
        image = np.array(Image.open(path + "/" + file))
        image = list(np.reshape(image, (1, 4096))[0])
        s.append(image)
    return s

Jetzt fangen wir mit unsere Hauptkomponenten Analyse an, zuerst zentralisieren wir die Dateien, indem jede Demension/Merkmale (entspricht zur Spalte) die durchschnittliche Demension substraiert, damit jede Demension Durchschinitt 0 kriegt. 

In [2]:
def get_centralized(data):
    dataT = np.transpose(data)
    for i in range(len(dataT)):
        average = np.sum(dataT[i]) / len(dataT[i])
        for j in range(len(dataT[i])):
            j = j - average
    return np.transpose(dataT)

Und berechnen wir die Kovarianzmatrix.

In [3]:
def mycovariance(matrix):
    middlevec = get_average(matrix)
    length = len(matrix)
    matrix1 = np.transpose(np.subtract(matrix, middlevec))
    matrix2 = np.transpose(matrix1)
    matrix = np.dot(matrix1, matrix2)
    return np.divide(matrix, length-1)

def get_average(images):
    result = np.zeros((len(images[0])), dtype=np.float)
    for pixel in range(0, len(images[0])):
        sum = 0
        for image in range(0, len(images)):
            sum += images[image][pixel]
        result[pixel] = (sum / len(images))
    return result

Berechne die Eigenwert und Eigenvektor der Kovarianz Matirx von der zentralisierten Datei, und sortieren die
 Eigenwert, damit wir deren entsprechende k-größte Eigenvektoren bekommen, solche Vektoren sind am meinsten 
 mit einander unabhängig.

In [4]:
def get_top_k(data, k):
    value, vector = np.linalg.eig(data)
    sorted_indices = np.argsort(value)  # sortierte Index
    k_vector = vector[:,sorted_indices[:-k-1:-1]]
    return k_vector

Multilplizieren die originale Datei(m*n) und die berechnete k-Vektoren(n*k), sodass kriegen wir
die reduzierte neue Datei mit (m*k) Matrix.

In [5]:
def to_Kdimension(data, k):
    data_central = get_centralized(data)
    cov = mycovariance(data_central)
    kVec = get_top_k(cov, k)
    return np.dot(data, kVec)

In [6]:
pictures = get_file()
new_pic = np.transpose(to_Kdimension(pictures, k=9))

print(np.shape(new_pic))

(9, 13233)


Jetzt reduzieren wir den Datensatz nur mit Demensionsanzahl 9.